In [23]:
from __future__ import division
from bokeh.io import output_notebook, show,output_file
from bokeh.plotting import figure
import numpy as np
import pandas as pd 
from pandas import DataFrame, read_csv
from bokeh.models import ColumnDataSource
import math
from bokeh.models.annotations import Label

#choose how o deal with missing values
        #mode=0 for ignoring missing values
        #mode=1 for taking median across attribute
        #mode=2 for taking mean across attribute
mode=2
dict1=[]    
output_notebook()

#import dataset
file = r'"Wholesale customers data.csv"'
df= pd.read_csv("Wholesale customers data.csv")
source=ColumnDataSource(df)
#print (df.head())
total_rows = len(df.index)

#import dataset with mising values
file2 = r'"Wholesale customers data-missing.csv"'
df_missing = pd.read_csv("Wholesale customers data-missing.csv")



    

figures=[]
#print (df.columns[1])
column_list = df.columns
#global variable


def func(inout,x1,y1):
    small_df = df_missing[[x1,y1]]
    cleaned_df = small_df.dropna(how='all')                      #if both the values are mssing, delete the row.
    '''dict2=[]'''
    global dict1
    dict1.clear()
    if(inout==0):
         #trimming dataset, include rows with missing data only
        cleaned_df= cleaned_df[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()] 
        #drop all rows => ignoring rows with missing values
        cleaned_df = cleaned_df.dropna()
        index_list=cleaned_df.index[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()].tolist()  
        #list of actual values
        row_containing_missing_value_x=[]
        row_containing_missing_value_y=[]
        for j in range(0,len(index_list)):
            row_containing_missing_value_x.append(df.loc[index_list[j],x1]) 
            row_containing_missing_value_y.append(df.loc[index_list[j],y1])
        dict1.append(row_containing_missing_value_x)
        dict1.append(row_containing_missing_value_y)
        #dict1=dict2
        # dict1 = [[row_containing_missing_value_x],[row_containing_missing_value_y]]
        return cleaned_df
        
    if (inout==1):
        x1_missing_median =cleaned_df.loc[:,x1].median()                 #calculating median
        y1_missing_median =cleaned_df.loc[:,y1].median()
        index_list=cleaned_df.index[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()].tolist()    
         #trimming dataset, include rows with missing data only
        cleaned_df= cleaned_df[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()]         
        #print (cleaned_df.head())
        #fill missing values
        cleaned_df[x1].fillna(x1_missing_median,inplace=True)
        cleaned_df[y1].fillna(y1_missing_median,inplace=True)
        #list of actual values
        row_containing_missing_value_x=[]
        row_containing_missing_value_y=[]
        for j in range(0,len(index_list)):
            row_containing_missing_value_x.append(df.loc[index_list[j],x1]) 
            row_containing_missing_value_y.append(df.loc[index_list[j],y1])
        dict2.append(row_containing_missing_value_x)
        dict2.append(row_containing_missing_value_y)   
        dict1=dict2
    # dict1 = [row_containing_missing_value_x,row_containing_missing_value_y]
        return cleaned_df
    
    if (inout==2):
        x1_missing_mean =cleaned_df.loc[:,x1].mean()                 #calculating mean
        y1_missing_mean =cleaned_df.loc[:,y1].mean()
        index_list=cleaned_df.index[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()].tolist()    
         #trimming dataset, include rows with missing data only
        cleaned_df= cleaned_df[cleaned_df[x1].isnull() | cleaned_df[y1].isnull()]         
        #print (cleaned_df.head())
        #fill missing values
        cleaned_df[x1].fillna(x1_missing_mean,inplace=True)
        cleaned_df[y1].fillna(y1_missing_mean,inplace=True)
        #list of actual values
        row_containing_missing_value_x=[]
        row_containing_missing_value_y=[]
        for j in range(0,len(index_list)):
            row_containing_missing_value_x.append(df.loc[index_list[j],x1]) 
            row_containing_missing_value_y.append(df.loc[index_list[j],y1])
        dict1.append(row_containing_missing_value_x)
        dict1.append(row_containing_missing_value_y)
        #dict1=dict2
        #dict1 = [[row_containing_missing_value_x],[row_containing_missing_value_y]]
        return cleaned_df
        

for it_1 in range (2,4):
    for it_2 in range (it_1+1,8):
        ##regression
        #x1='Fresh'
        #y1='Detergents_Paper'
        x1 = column_list[it_1]
        y1 = column_list[it_2]
        
        #print (x1+" "+y1)
        x1_mean = df.loc[:,x1].mean()
        y1_mean = df.loc[:,y1].mean()
        num=0
        den=0
        for i in range(1,total_rows):
            num += ((df.loc[i,x1])-x1_mean) *((df.loc[i,y1])-y1_mean)
            den += math.pow(((df.loc[i,x1])-x1_mean),2)
        b=0
        a=0
        b = num/den
        a = y1_mean - (b*x1_mean)
        #print ("%s , %s",a,b)
        fx=[]
        for i in range(0,total_rows):
            f = a + b*(df.loc[i,x1])
            fx.append(f)
        list_y = [a+b*x for x in df.loc[:,x1]]
        
        mode_str=''
        if(mode==0): mode_str='neglect missing values'
        if (mode==1): mode_str='Median across attribute'
        if(mode==2) : mode_str='Mean across attribute'
        #calling function func to deal with missing values
        cleaned_df = func(mode,x1,y1)
          
        #print (dict1)
        TOOLS = "pan,box_select,lasso_select,box_zoom,wheel_zoom,save,reset,help"
        s1=figure(plot_height=200, plot_width=500,tools=TOOLS)
        s1.scatter(x=x1,y=y1,source=source,color='orange', alpha=0.3,legend='Normal data')
        s1.diamond(x=x1,y=y1,source=cleaned_df,fill_color='navy',line_color='blue',size=10, fill_alpha=0.9)
        s1.square(x=dict1[0],y=dict1[1],legend='orignal missing values',color = 'red',size=5,alpha=1)
        s1.line(x=x1,y=list_y,source=source,color='green',line_width=2,legend='linear regression line')
        s1.xaxis.axis_label = x1 + ', predicted missing values in diamond navy'
        s1.yaxis.axis_label = y1
        label = Label(x=200, y=2000, x_offset=140,y_offset=132, text="mode="+mode_str, text_baseline="middle")
        s1.add_layout(label)

        figures.append(s1)
        show(s1)

    
    

#show(figures[0:2])
#p=gridplot([[s1,s2],[s3],[figures[0],figures[1]],[figures[2],figures[2]],[figures[3],figures[4]]])
#p=gridplot([figures[0],figures[1]])
#q=gridplot([[figures[0],figures[6]]])
#show(p)
#show(q)

Loading BokehJS ...

[[12356], [6036]]


C:\Users\abhishek\Anaconda\envs\py36\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


[[20398, 955], [3, 6536]]


C:\Users\abhishek\Anaconda\envs\py36\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


...

C:\Users\abhishek\Anaconda\envs\py36\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


[[6036, 3243, 3944], [1382, 761, 523]]


C:\Users\abhishek\Anaconda\envs\py36\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)


[[6036, 1703], [2794, 1153]]


C:\Users\abhishek\Anaconda\envs\py36\lib\site-packages\bokeh\util\deprecation.py:34: BokehDeprecationWarning: 
Supplying a user-defined data source AND iterable values to glyph methods is deprecated.

See https://github.com/bokeh/bokeh/issues/2056 for more information.

  warn(message)
